In [1]:
! pip install torch transformers pillow opencv-python


In [2]:
import cv2
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification

# Load the processor and model
processor = AutoImageProcessor.from_pretrained("dima806/hand_gestures_image_detection")
model = AutoModelForImageClassification.from_pretrained("dima806/hand_gestures_image_detection")


d:\workspace\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
labels = ["palm", "fist", "thumb_up", "thumb_down", "victory", "okay"]

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame to a PIL image
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Preprocess the image
    inputs = processor(images=image, return_tensors="pt")

    # Perform the prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class
    predicted_class = outputs.logits.argmax(-1).item()
    
    if predicted_class < len(labels):
        predicted_label = labels[predicted_class]
    else:
        predicted_label = "Unknown"
    # Display the result on the frame
    cv2.putText(frame, f"Gesture: {predicted_label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Hand Gesture Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()